In [9]:
import os 
import json
import requests
import numpy as np
import pandas as pd
from django.http.response import JsonResponse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
tmp_my_ingredient = ['오이','양파','부추']

In [37]:
col = 'ingredient'
my_ingredinet = pd.DataFrame(data={col:[' '.join(tmp_my_ingredient)]})
my_ingredinet

,ingredient
0,오이 양파 부추


In [38]:
DJANGO_ORIGIN = 'http://localhost:8000'
url = DJANGO_ORIGIN + '/static/recipe.json'
res = requests.get(url).json()
recipe_df = pd.DataFrame.from_dict(res)

In [39]:
recipe_df

,recipe_info_id,view,ingredient
0,420777,252299,코다리 고추
1,667367,116104,부추 양파 당근 고추 가루 가루 얼음
2,682697,327176,돼지고기 고추
3,690101,171511,마늘 간장 물엿 소금 후추 참기름 깨
4,693694,110973,삼겹살 대파 물 양파 마늘 후추 파 월계수
...,...,...,...
491,5816092,236561,차돌박이 숙주나물 달래 고추
492,5817718,134708,돼지고기 메추리알 고추
493,5821585,230254,불고기 당면 양파 대파 버섯 당근
494,5840021,114278,낙지 대파 애호박 소금


In [40]:
tmp = recipe_df[['ingredient']].append(my_ingredinet, ignore_index=True)

In [41]:
tmp

,ingredient
0,코다리 고추
1,부추 양파 당근 고추 가루 가루 얼음
2,돼지고기 고추
3,마늘 간장 물엿 소금 후추 참기름 깨
4,삼겹살 대파 물 양파 마늘 후추 파 월계수
...,...
492,돼지고기 메추리알 고추
493,불고기 당면 양파 대파 버섯 당근
494,낙지 대파 애호박 소금
495,대파


In [42]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
count_vect = count_vect.fit(recipe_df['ingredient'])

In [43]:
ing_mat = count_vect.transform(tmp['ingredient'])

In [44]:
ing_sim = cosine_similarity(ing_mat, ing_mat)
ing_sim_sorted_ind = ing_sim.argsort()[:,::-1]

In [45]:
top_n = 3   # 프론트와 논의 필요
similar_indexes = ing_sim_sorted_ind[tmp.shape[0]-1, 1:2*top_n]     # tmp.shape[0]-1 = 맨 마지막 행
similar_recipes = recipe_df.iloc[similar_indexes].sort_values('view', ascending=False)[:top_n]

In [46]:
similar_recipes_id = []
for r in similar_recipes['recipe_info_id']:
    similar_recipes_id.append(r)

In [47]:
similar_recipes_id
# 토마토 가지 계란 : [4025350, 5046976, 5444973]
# 토마토 계란 : [4015267, 4763537, 4655671]
# 돼지고기 김치 :
# 새우 계란 
# 식빵 참치
# 참치 김치

[5421184, 3687411, 5773859]

In [48]:
cond = recipe_df['ingredient'].str.contains('가루')
recipe_df[cond]

,recipe_info_id,view,ingredient
1,667367,116104,부추 양파 당근 고추 가루 가루 얼음
16,991943,119928,김치 두부 가루 계란 대파 식용유
38,1775942,106753,가루 설탕 물
89,3031684,145401,두부 버섯 양파 대파 가루 녹말 참기름 깨 마늘 소금
93,3083491,105020,멸치 다시마 동태 무 대파 고추가루 물
94,3085196,129548,멸치 다시마 가루 식용유 감자 양파 호박 고추 고추
102,3230695,147762,적두 가루 쌀
122,3416940,103707,가루 막걸리
127,3475852,204786,무 배 양파 대파 마늘 생강 가루 고추 고추
131,3516293,162351,돼지고기 김치 고추가루 육수 김치 육수
